In [1]:
import os
import openai
import nest_asyncio
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore # pip install llama-index-vector-stores-chroma
from llama_index.core import StorageContext

In [2]:
nest_asyncio.apply()

In [3]:
# https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings/
# https://docs.llamaindex.ai/en/stable/examples/embeddings/OpenAI/

openai.api_key = os.environ["OPENAI_API_KEY"]
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.embed_model = embed_model

In [4]:
# https://docs.llamaindex.ai/en/stable/understanding/storing/storing/
# https://docs.llamaindex.ai/en/latest/examples/vector_stores/chroma_metadata_filter/

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# if you want to delete a pre-existing collection, do this:
#db.delete_collection(name="Your_collection_name")
# create collection
chroma_collection = db.get_or_create_collection("Your_collection_name")

In [5]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

# load some documents
documents = SimpleDirectoryReader("./rag_docs").load_data()

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# configure text splitter
text_splitter = SentenceSplitter(chunk_size=1000, chunk_overlap=10)
Settings.text_splitter = text_splitter


# create your index
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    transformations=[text_splitter]
)

In [ ]:
# check if your vector store is working; create a query engine and query
query_engine = index.as_query_engine()
response = query_engine.query("What's a Paca?'")
print(response)

In [7]:
# Use this to insert new documents after the store has been created

index = VectorStoreIndex([])
for doc in documents:
    index.insert(doc)

In [9]:
# https://docs.llamaindex.ai/en/stable/examples/chat_engine/chat_engine_openai/

# Check that you can use your vectore store to chat with an chat completion model; first, create the chat engine"

llm = OpenAI(model="gpt-4o")

system_prompt = 'You really like Pacas. You know a lot about them.'

chat_engine = index.as_chat_engine(
    chat_mode="openai",
    llm=llm,
    system_prompt = system_prompt,
    verbose=True
)

In [ ]:
# then use the chat engine to have chat about whatever is in your vector storage

response = chat_engine.chat(
    "Use the tool to answer: what are Pacas?"
)


In [ ]:
# Print the response only

response.response